In [ ]:
import os
import cv2
import numpy as np
from collections import defaultdict
from scipy.signal import savgol_filter

# Root input directories
ANNOTATIONS_DIR = r"F:\ml class project\VisDrone2019-DET-test-challenge\VisDrone2019-MOT-train\annotations"
SEQUENCES_DIR   = r"F:\ml class project\VisDrone2019-DET-test-challenge\VisDrone2019-MOT-train\sequences"

# Output base folders: one for frames and one for tracklet CSV files
FRAMES_BASE   = "frame_data"
TRACKLETS_BASE = "tracklet_data"
os.makedirs(FRAMES_BASE, exist_ok=True)
os.makedirs(TRACKLETS_BASE, exist_ok=True)

# Vehicle categories: car=4, van=5, bus=6, truck=7.
VEHICLE_CATEGORIES = {4, 5, 6, 7}

# Visualization settings
TRAJECTORY_COLOR = (0, 0, 255)  # Red (BGR)
BOX_COLOR        = (0, 255, 0)  # Green (BGR)
LINE_THICKNESS   = 2
MAX_HISTORY      = 500

# -----------------------------------------------------------------------------
# Helper: Smooth trajectory using Savitzky-Golay filter
# -----------------------------------------------------------------------------
def smooth_trajectory(points, window=11, poly=2):
    """
    Applies the Savitzky-Golay filter for an ultra-smooth trajectory.
    Returns a list of smoothed (x,y) coordinates.
    """
    if len(points) < window:
        return points  # Not enough points to smooth
    
    x_vals, y_vals = zip(*points)
    w_len = min(window, len(x_vals))
    
    smooth_x = savgol_filter(x_vals, window_length=w_len, polyorder=poly)
    smooth_y = savgol_filter(y_vals, window_length=w_len, polyorder=poly)
    
    return list(zip(map(int, smooth_x), map(int, smooth_y)))

# -----------------------------------------------------------------------------
# Process a single sequence: extract frames and tracklets data
# -----------------------------------------------------------------------------
def process_sequence(seq_name):
    """
    For one sequence folder (seq_name), this function:
      1) Reads images from SEQUENCES_DIR/seq_name.
      2) Reads annotations from ANNOTATIONS_DIR/seq_name + ".txt".
      3) Filters annotations for the categories: car, van, bus, truck.
      4) Draws bounding boxes and smoothed trajectories with random cuts.
      5) Saves processed frames in FRAMES_BASE/seq_name.
      6) Writes out the tracklets CSV file in TRACKLETS_BASE/seq_name.
    """
    # Construct paths for images and annotation
    image_folder = os.path.join(SEQUENCES_DIR, seq_name)
    annot_file   = os.path.join(ANNOTATIONS_DIR, seq_name + ".txt")
    
    if not os.path.exists(annot_file):
        print(f"[WARNING] Annotation file missing for sequence: {seq_name}")
        return

    # Create subfolder for frame outputs for this sequence
    seq_frames_folder = os.path.join(FRAMES_BASE, seq_name)
    os.makedirs(seq_frames_folder, exist_ok=True)
    
    # Create subfolder for tracklet CSV output for this sequence
    seq_tracklets_folder = os.path.join(TRACKLETS_BASE, seq_name)
    os.makedirs(seq_tracklets_folder, exist_ok=True)
    tracklet_file = os.path.join(seq_tracklets_folder, f"tracklets_{seq_name}.csv")
    
    # Read annotation and filter by the defined vehicle categories
    object_tracks = defaultdict(list)
    with open(annot_file, "r") as f:
        for line in f:
            values = list(map(int, line.strip().split(',')))
            if len(values) < 10:
                continue
            frame_id, obj_id, x, y, w, h, conf, category, vis, unused = values
            
            if category in VEHICLE_CATEGORIES:
                cx = x + w // 2
                cy = y + h // 2
                object_tracks[obj_id].append((frame_id, x, y, w, h, cx, cy))
    
    # Generate random cut ranges for trajectories (optional)
    trajectories = defaultdict(list)
    cut_ranges_per_object = {}
    for obj_id in object_tracks.keys():
        num_cuts = np.random.randint(4, 6)  # 4 or 5 cuts
        total_frames = len(object_tracks[obj_id])
        if total_frames > 70:
            cut_positions = sorted(
                np.random.choice(range(30, total_frames - 30), num_cuts, replace=False)
            )
            cut_ranges = [
                (cut_positions[i], cut_positions[i] + np.random.randint(20, 40))
                for i in range(num_cuts)
            ]
        else:
            cut_ranges = []
        cut_ranges_per_object[obj_id] = cut_ranges
    
    # Get and sort all unique frame IDs from the annotation
    all_frame_ids = sorted(set(f_id for obj in object_tracks.values() for f_id, *_ in obj))
    print(f"Processing sequence: {seq_name}")
    print(f"  Total unique frame IDs: {len(all_frame_ids)}")
    
    saved_count = 0
    # Process each frame of the sequence
    for frame_id in all_frame_ids:
        frame_file = f"{frame_id:07d}.jpg"
        image_path = os.path.join(image_folder, frame_file)
        if not os.path.exists(image_path):
            print(f"[WARNING] Missing file: {image_path}")
            continue
        
        img = cv2.imread(image_path)
        if img is None:
            continue
        
        # Draw bounding boxes and update trajectories for each object
        for obj_id, frames_data in object_tracks.items():
            # Create a mapping: frame_id -> (x, y, w, h, cx, cy)
            obj_data_map = {d[0]: (d[1], d[2], d[3], d[4], d[5], d[6]) for d in frames_data}
            if frame_id in obj_data_map:
                x, y, w, h, cx, cy = obj_data_map[frame_id]
                trajectories[obj_id].append((cx, cy))
                if len(trajectories[obj_id]) > MAX_HISTORY:
                    trajectories[obj_id].pop(0)
                # Draw the bounding box and label on the image
                cv2.rectangle(img, (x, y), (x + w, y + h), BOX_COLOR, 2)
                cv2.putText(img, f"Obj {obj_id}",
                            (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, BOX_COLOR, 1)
        
        # Draw smoothed trajectories with random cuts
        for obj_id, traj_points in trajectories.items():
            if len(traj_points) > 2:
                smoothed_traj = smooth_trajectory(traj_points)
                if len(smoothed_traj) > 1:
                    int_traj = np.array(smoothed_traj)
                    prev_pt = None
                    for i, point in enumerate(int_traj):
                        if any(start <= i <= end for start, end in cut_ranges_per_object[obj_id]):
                            prev_pt = None
                        else:
                            if prev_pt is not None:
                                cv2.line(img, prev_pt, tuple(point), TRAJECTORY_COLOR, LINE_THICKNESS)
                            prev_pt = tuple(point)
        
        # Save the processed frame in the sequence's frame folder
        out_path = os.path.join(seq_frames_folder, frame_file)
        cv2.imwrite(out_path, img)
        saved_count += 1
    
    # cv2.destroyAllWindows()
    print(f"  Processed {seq_name}: Saved {saved_count} frames to {seq_frames_folder}")

    # Write out the tracklets CSV for this sequence
    with open(tracklet_file, "w") as f_out:
        f_out.write("obj_id,frame_id,x,y,w,h,cx,cy\n")
        for obj_id, data_list in object_tracks.items():
            data_sorted = sorted(data_list, key=lambda d: d[0])
            for (frame_id, x, y, w, h, cx, cy) in data_sorted:
                f_out.write(f"{obj_id},{frame_id},{x},{y},{w},{h},{cx},{cy}\n")
    print(f"  Tracklets CSV for {seq_name} saved to {tracklet_file}")
    print("--------------------------------------------------")

# -----------------------------------------------------------------------------
# Main Loop: Iterate over all sequence directories
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    seq_folders = sorted(d for d in os.listdir(SEQUENCES_DIR)
                         if os.path.isdir(os.path.join(SEQUENCES_DIR, d)))
    print(f"Found {len(seq_folders)} sequence folders.")
    for seq_name in seq_folders:
        process_sequence(seq_name)
    
    print("All sequences processed!")



In [ ]:
import os
import csv
import numpy as np
from collections import defaultdict

annotations_folder = r"F:\ml class project\VisDrone2019-DET-test-challenge\VisDrone2019-MOT-train\annotations"
output_folder = r"F:\ml class project\VisDrone2019-DET-test-challenge\annotation and ground truth"

VEHICLE_CATEGORIES = {4, 5, 6, 7}

def process_annotation_file(annot_file, output_folder):
    base_name = os.path.splitext(os.path.basename(annot_file))[0]

    object_data = defaultdict(list)

    # Load annotations
    with open(annot_file, "r") as f:
        for line in f:
            vals = list(map(int, line.strip().split(',')))
            frame_id, obj_id, x, y, w, h, _, category, _, _ = vals
            if category in VEHICLE_CATEGORIES:
                object_data[obj_id].append((frame_id, x, y, w, h))

    new_tracklet_id = 0
    tracklet_annotations = []
    tracklet_merge_map = []

    for obj_id, detections in object_data.items():
        detections.sort()
        total_frames = len(detections)

        # Create 4 to 5 gaps (simulate occlusions)
        num_cuts = np.random.randint(4, 6) if total_frames > 70 else 0
        cut_points = sorted(np.random.choice(range(10, total_frames - 10), num_cuts, replace=False)) if num_cuts > 0 else []
        cut_points.append(total_frames)

        prev_cut = 0
        for cut in cut_points:
            segment = detections[prev_cut:cut]
            for det in segment:
                frame_id, x, y, w, h = det
                tracklet_annotations.append([frame_id, x, y, w, h, new_tracklet_id])
            tracklet_merge_map.append((obj_id, new_tracklet_id))
            new_tracklet_id += 1
            prev_cut = cut

    # Save annotations with tracklet IDs
    output_csv = os.path.join(output_folder, f"{base_name}_tracklets.csv")
    with open(output_csv, "w", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["frame_id", "x", "y", "w", "h", "tracklet_id"])
        writer.writerows(tracklet_annotations)

    # Save ground truth merging info
    merge_csv = os.path.join(output_folder, f"{base_name}_gt_merge.csv")
    with open(merge_csv, "w", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["obj_id", "tracklet_id"])
        writer.writerows(tracklet_merge_map)

    print(f"Processed {annot_file}, created {output_csv} and {merge_csv}")


# Process all .txt files
os.makedirs(output_folder, exist_ok=True)
for file in os.listdir(annotations_folder):
    if file.endswith(".txt"):
        file_path = os.path.join(annotations_folder, file)
        process_annotation_file(file_path, output_folder)


In [ ]:
import os
import glob

def filter_annotations(folder_path):
    """
    Process all annotation files in the given folder, keeping only categories 4,5,6
    and saving the modified files in place.
    
    Args:
        folder_path: Path to the folder containing annotation files
    """
    # Get all .txt files in the folder
    annotation_files = glob.glob(os.path.join(folder_path, "*.txt"))
    
    for file_path in annotation_files:
        # Read the original file
        with open(file_path, 'r') as f:
            lines = f.readlines()
        
        # Filter lines (keep only categories 4,5,6)
        filtered_lines = []
        for line in lines:
            # Handle both comma and space separated
            values = line.strip().replace(',', ' ').split()
            if len(values) >= 8:  # Ensure we have enough columns
                try:
                    category = int(values[7])  # Category is 8th column (0-indexed 7)
                    if category in {4, 5, 6}:
                        filtered_lines.append(line)
                except (ValueError, IndexError):
                    continue
        
        # Write the filtered content back to the same file
        with open(file_path, 'w') as f:
            f.writelines(filtered_lines)
        
        print(f"Processed {file_path} - kept {len(filtered_lines)}/{len(lines)} lines")

if __name__ == "__main__":
    # Example usage:
    folder_path = input("Enter the path to your annotations folder: ").strip()
    if os.path.exists(folder_path):
        print(f"Processing files in {folder_path}")
        filter_annotations(folder_path)
        print("Processing complete!")
    else:
        print(f"Error: Folder not found - {folder_path}")

In [11]:
import os
import glob

def cleanup_empty_annotations_and_sequences(annotations_path, sequences_path):
    """
    Clean up empty annotation files and their corresponding sequence folders
    
    Args:
        annotations_path: Path to annotations folder
        sequences_path: Path to sequences folder
    """
    # Get all .txt files in the annotations folder
    annotation_files = glob.glob(os.path.join(annotations_path, "*.txt"))
    
    deleted_annotations = 0
    deleted_sequences = 0
    
    for annot_file in annotation_files:
        # Check if file is empty
        if os.path.getsize(annot_file) == 0:
            # Get the base filename without extension
            base_name = os.path.splitext(os.path.basename(annot_file))[0]
            
            # Delete the annotation file
            os.remove(annot_file)
            deleted_annotations += 1
            print(f"Deleted empty annotation file: {annot_file}")
            
            # Find and delete corresponding sequence folder
            seq_folder = os.path.join(sequences_path, base_name)
            if os.path.exists(seq_folder):
                try:
                    # Remove all files in the sequence folder first
                    for file in os.listdir(seq_folder):
                        file_path = os.path.join(seq_folder, file)
                        try:
                            if os.path.isfile(file_path):
                                os.unlink(file_path)
                        except Exception as e:
                            print(f"Failed to delete {file_path}. Reason: {e}")
                    
                    # Remove the empty directory
                    os.rmdir(seq_folder)
                    deleted_sequences += 1
                    print(f"Deleted corresponding sequence folder: {seq_folder}")
                except Exception as e:
                    print(f"Failed to delete sequence folder {seq_folder}. Reason: {e}")
    
    print("\nCleanup summary:")
    print(f"Deleted {deleted_annotations} empty annotation files")
    print(f"Deleted {deleted_sequences} corresponding sequence folders")

if __name__ == "__main__":
    print("Annotation and Sequence Folder Cleanup Tool")
    print("------------------------------------------\n")
    
    # Get user input for paths
    annotations_path = input("Enter path to your ANNOTATIONS folder: ").strip()
    sequences_path = input("Enter path to your SEQUENCES folder: ").strip()
    
    # Validate paths
    if not os.path.exists(annotations_path):
        print(f"\nError: Annotations folder not found at {annotations_path}")
    elif not os.path.exists(sequences_path):
        print(f"\nError: Sequences folder not found at {sequences_path}")
    else:
        print("\nStarting cleanup process...")
        cleanup_empty_annotations_and_sequences(annotations_path, sequences_path)
        print("\nCleanup complete!")

Annotation and Sequence Folder Cleanup Tool
------------------------------------------


Starting cleanup process...

Cleanup summary:
Deleted 0 empty annotation files
Deleted 0 corresponding sequence folders

Cleanup complete!


In [9]:
import os
import random
import csv
from collections import defaultdict

def process_annotation_file(input_path, output_dir):
    """Process a single annotation file and generate three output files"""
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Create output filenames
    base_name = os.path.splitext(os.path.basename(input_path))[0]
    refined_file = os.path.join(output_dir, f"{base_name}_refined.txt")
    tracklets_file = os.path.join(output_dir, f"{base_name}_tracklets.csv")
    ground_truth_file = os.path.join(output_dir, f"{base_name}_ground_truth.csv")

    # Read and parse the input file
    try:
        with open(input_path, 'r') as f:
            lines = [line.strip() for line in f if line.strip()]
    except Exception as e:
        print(f"Error reading {input_path}: {e}")
        return

    # First pass: Filter and organize annotations
    original_tracks = defaultdict(list)
    for line in lines:
        try:
            parts = line.split()
            if len(parts) >= 10:  # Ensure we have all required fields
                category = int(parts[7])
                if category in {4, 5, 6}:  # Only keep categories 4,5,6
                    original_tracks[int(parts[1])].append({
                        'frame_id': int(parts[0]),
                        'original_id': int(parts[1]),
                        'bbox': list(map(int, parts[2:6])),
                        'score': float(parts[6]),
                        'category': category,
                        'visibility': float(parts[8]),
                        'ignored': int(parts[9]),
                        'raw_line': line
                    })
        except (ValueError, IndexError) as e:
            print(f"Skipping malformed line: {line} ({e})")
            continue

    # Initialize output files
    with open(refined_file, 'w') as refined_f, \
         open(tracklets_file, 'w') as tracklets_f, \
         open(ground_truth_file, 'w') as truth_f:

        # Write CSV headers
        tracklet_writer = csv.writer(tracklets_f)
        tracklet_writer.writerow([
            'frame_id', 'tracklet_id', 'original_id',
            'x', 'y', 'width', 'height',
            'category', 'visibility', 'ignored'
        ])

        truth_writer = csv.writer(truth_f)
        truth_writer.writerow(['original_id', 'tracklet_ids'])

        global_track_id = 1
        track_mapping = defaultdict(list)

        # Process each original track
        for orig_id, track in original_tracks.items():
            if not track:
                continue

            # Sort by frame_id
            track.sort(key=lambda x: x['frame_id'])
            

            # Determine number of splits (1-3 for long tracks)
            num_splits = random.randint(1, 3) if len(track) > 30 else 1

            if num_splits == 1:
                # Keep as single tracklet
                new_id = global_track_id
                global_track_id += 1

                for ann in track:
                    # Write to refined file (update track ID)
                    parts = ann['raw_line'].split()
                    parts[1] = str(new_id)
                    refined_f.write(' '.join(parts) + '\n')

                    # Write to tracklets CSV
                    tracklet_writer.writerow([
                        ann['frame_id'], new_id, orig_id,
                        ann['bbox'][0], ann['bbox'][1],
                        ann['bbox'][2], ann['bbox'][3],
                        ann['category'], ann['visibility'], ann['ignored']
                    ])

                track_mapping[orig_id].append(new_id)
            else:
                # Split into multiple tracklets
                split_points = sorted(random.sample(
                    range(15, len(track)-15),
                    num_splits
                ))
                split_points = [0] + split_points + [len(track)]

                for i in range(len(split_points)-1):
                    start, end = split_points[i], split_points[i+1]
                    if end - start < 10:  # Skip short segments
                        continue

                    # Create gap (15-60 frames)
                    gap_start = random.randint(start, end-1)
                    gap_end = min(gap_start + random.randint(15, 60), end)

                    # Create new tracklet
                    new_id = global_track_id
                    global_track_id += 1

                    for j in range(start, end):
                        if gap_start <= j < gap_end:
                            continue  # Skip frames in the gap

                        ann = track[j]
                        # Write to refined file (update track ID)
                        parts = ann['raw_line'].split()
                        parts[1] = str(new_id)
                        refined_f.write(' '.join(parts) + '\n')

                        # Write to tracklets CSV
                        tracklet_writer.writerow([
                            ann['frame_id'], new_id, orig_id,
                            ann['bbox'][0], ann['bbox'][1],
                            ann['bbox'][2], ann['bbox'][3],
                            ann['category'], ann['visibility'], ann['ignored']
                        ])

                    track_mapping[orig_id].append(new_id)

        # Write ground truth mapping
        for orig_id, tracklet_ids in track_mapping.items():
            truth_writer.writerow([orig_id, ' '.join(map(str, tracklet_ids))])

    print(f"Processed {input_path} -> {base_name}_* files")

def process_directory(input_dir, output_dir):
    """Process all annotation files in a directory"""
    if not os.path.exists(input_dir):
        print(f"Error: Input directory not found: {input_dir}")
        return

    print(f"Processing files in {input_dir}...")
    
    # Process each annotation file
    processed_files = 0
    for filename in os.listdir(input_dir):
        if filename.endswith('.txt'):
            input_path = os.path.join(input_dir, filename)
            process_annotation_file(input_path, output_dir)
            processed_files += 1

    print(f"\nProcessing complete! {processed_files} files processed.")
    print(f"Output saved to: {output_dir}")

if __name__ == "__main__":
    # User input
    input_dir = input("Enter path to annotations directory: ").strip()
    output_dir = input("Enter output directory path: ").strip()

    # Process the directory
    process_directory(input_dir, output_dir)

Error: Input directory not found: 


In [31]:
# import os
# import pandas as pd
# import numpy as np
# import random

# def load_annotation_file(file_path):
#     """
#     Load a VisDrone annotation file into a DataFrame with proper column names.
#     """
#     column_names = [
#         'frame', 'object_id', 'bbox_left', 'bbox_top', 'bbox_width', 'bbox_height',
#         'score', 'object_category', 'truncation', 'occlusion'
#     ]
#     try:
#         df = pd.read_csv(file_path, header=None, names=column_names)
#         return df
#     except Exception as e:
#         print(f"Error reading file {file_path}: {e}")
#         return None

# def get_continuous_segments(frames):
#     """
#     Identify continuous frame segments for an object.
#     Returns a list of tuples (start_frame, end_frame, length).
#     """
#     if len(frames) == 0:
#         return []
    
#     frames = np.sort(frames)
#     segments = []
#     start = frames[0]
#     prev = frames[0]
    
#     for curr in frames[1:]:
#         if curr != prev + 1:
#             segments.append((start, prev, prev - start + 1))
#             start = curr
#         prev = curr
    
#     segments.append((start, prev, prev - start + 1))
#     return segments

# def remove_middle_frames(df, object_id, min_remove=15, max_remove=60, min_frames=120):
#     """
#     Remove a continuous block of frames from the middle of a continuous trajectory
#     for a given object if it has a segment with more than min_frames frames.
#     """
#     obj_df = df[df['object_id'] == object_id].sort_values('frame')
#     frames = obj_df['frame'].values
    
#     # Find continuous segments
#     segments = get_continuous_segments(frames)
    
#     # Filter segments with more than min_frames
#     long_segments = [s for s in segments if s[2] > min_frames]
    
#     if not long_segments:
#         return df  # No continuous segments long enough
    
#     # Select the longest segment to modify
#     target_segment = max(long_segments, key=lambda x: x[2])
#     start_frame, end_frame, segment_length = target_segment
    
#     # Calculate middle third of the segment (by frame numbers, not indices)
#     segment_frames = np.arange(start_frame, end_frame + 1)
#     middle_start = start_frame + (segment_length // 3)
#     middle_end = start_frame + (segment_length * 2 // 3)
    
#     # Available frames in the middle third
#     middle_frames = segment_frames[(segment_frames >= middle_start) & (segment_frames <= middle_end)]
    
#     if len(middle_frames) < min_remove:
#         return df  # Not enough frames to remove
    
#     # Decide number of frames to remove
#     num_to_remove = random.randint(min_remove, min(max_remove, len(middle_frames)))
    
#     # Select a continuous block of frames
#     max_start_idx = len(middle_frames) - num_to_remove
#     start_idx = random.randint(0, max_start_idx)
#     frames_to_remove = middle_frames[start_idx:start_idx + num_to_remove]
    
#     print(f"Removing {len(frames_to_remove)} frames ({frames_to_remove[0]}–{frames_to_remove[-1]}) "
#           f"from object {object_id} (segment {start_frame}–{end_frame}, {segment_length} frames)")
    
#     # Drop those frames from the original DataFrame
#     df = df[~((df['object_id'] == object_id) & (df['frame'].isin(frames_to_remove)))]
#     return df

# def process_folder(input_dir, output_dir):
#     """
#     Process all VisDrone annotation `.txt` files in a folder.
#     """
#     os.makedirs(output_dir, exist_ok=True)
#     annotation_files = [f for f in os.listdir(input_dir) if f.endswith('.txt')]

#     if not annotation_files:
#         print(f"No annotation files found in {input_dir}")
#         return

#     for file_name in annotation_files:
#         file_path = os.path.join(input_dir, file_name)
#         df = load_annotation_file(file_path)

#         if df is None:
#             continue

#         print(f"\nProcessing {file_name}...")

#         object_ids = df['object_id'].unique()
#         for obj_id in object_ids:
#             # Check if the object has a continuous segment longer than 120 frames
#             obj_df = df[df['object_id'] == obj_id].sort_values('frame')
#             frames = obj_df['frame'].values
#             segments = get_continuous_segments(frames)
#             if any(segment[2] > 120 for segment in segments):
#                 df = remove_middle_frames(df, obj_id)

#         # Save modified file
#         output_path = os.path.join(output_dir, file_name.replace('.txt', '-modified.txt'))
#         df.to_csv(output_path, header=False, index=False)
#         print(f"Saved modified annotations to {output_path}")

# # === Example Usage ===
# input_folder = r"F:\ml class project\VisDrone2019-DET-test-challenge\VisDrone2019-MOT-train\annotations"
# output_folder = r"F:\ml class project\VisDrone2019-DET-test-challenge\annotations-modified"

# process_folder(input_folder, output_folder)

import os
import pandas as pd
import numpy as np
import random

def load_annotation_file(file_path):
    """
    Load a VisDrone annotation file into a DataFrame with proper column names.
    """
    column_names = [
        'frame', 'object_id', 'bbox_left', 'bbox_top', 'bbox_width', 'bbox_height',
        'score', 'object_category', 'truncation', 'occlusion'
    ]
    try:
        df = pd.read_csv(file_path, header=None, names=column_names)
        return df
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

def get_continuous_segments(frames):
    """
    Identify continuous frame segments for an object.
    Returns a list of tuples (start_frame, end_frame, length).
    """
    if len(frames) == 0:
        return []
    
    frames = np.sort(frames)
    segments = []
    start = frames[0]
    prev = frames[0]
    
    for curr in frames[1:]:
        if curr != prev + 1:
            segments.append((start, prev, prev - start + 1))
            start = curr
        prev = curr
    
    segments.append((start, prev, prev - start + 1))
    return segments

def remove_middle_frames(df, object_id, min_remove=20, max_remove=80, min_frames=80, max_deletions=2):
    """
    Remove multiple continuous blocks of frames from a trajectory to create more tracklets.
    """
    obj_df = df[df['object_id'] == object_id].sort_values('frame')
    frames = obj_df['frame'].values
    
    # Find continuous segments
    segments = get_continuous_segments(frames)
    
    # Filter segments with more than min_frames
    long_segments = [s for s in segments if s[2] > min_frames]
    
    if not long_segments:
        return df  # No continuous segments long enough
    
    deletions = 0
    modified_df = df.copy()
    
    for start_frame, end_frame, segment_length in long_segments:
        if deletions >= max_deletions:
            break
        
        segment_frames = np.arange(start_frame, end_frame + 1)
        third_length = segment_length // 3
        
        # Target multiple thirds (e.g., middle and later)
        for third_start in [start_frame + third_length, start_frame + 2 * third_length]:
            if deletions >= max_deletions:
                break
            
            middle_start = max(start_frame, third_start)
            middle_end = min(end_frame, third_start + third_length)
            middle_frames = segment_frames[(segment_frames >= middle_start) & (segment_frames <= middle_end)]
            
            if len(middle_frames) < min_remove:
                continue
            
            num_to_remove = random.randint(min_remove, min(max_remove, len(middle_frames)))
            max_start_idx = len(middle_frames) - num_to_remove
            start_idx = random.randint(0, max_start_idx)
            frames_to_remove = middle_frames[start_idx:start_idx + num_to_remove]
            
            print(f"Removing {len(frames_to_remove)} frames ({frames_to_remove[0]}–{frames_to_remove[-1]}) "
                  f"from object {object_id} (segment {start_frame}–{end_frame}, {segment_length} frames)")
            
            modified_df = modified_df[~((modified_df['object_id'] == object_id) & 
                                     (modified_df['frame'].isin(frames_to_remove)))]
            deletions += 1
    
    return modified_df

def process_folder(input_dir, output_dir):
    """
    Process all VisDrone annotation `.txt` files in a folder.
    """
    os.makedirs(output_dir, exist_ok=True)
    annotation_files = [f for f in os.listdir(input_dir) if f.endswith('.txt')]

    if not annotation_files:
        print(f"No annotation files found in {input_dir}")
        return

    for file_name in annotation_files:
        file_path = os.path.join(input_dir, file_name)
        df = load_annotation_file(file_path)

        if df is None:
            continue

        print(f"\nProcessing {file_name}...")

        object_ids = df['object_id'].unique()
        for obj_id in object_ids:
            # Check if the object has a continuous segment longer than min_frames
            obj_df = df[df['object_id'] == obj_id].sort_values('frame')
            frames = obj_df['frame'].values
            segments = get_continuous_segments(frames)
            if any(segment[2] > 80 for segment in segments):  # Lowered to 80 for more eligibility
                df = remove_middle_frames(df, obj_id)

        # Save modified file
        output_path = os.path.join(output_dir, file_name.replace('.txt', '-modified.txt'))
        df.to_csv(output_path, header=False, index=False)
        print(f"Saved modified annotations to {output_path}")

# === Example Usage ===
input_folder = r"F:\ml class project\VisDrone2019-DET-test-challenge\VisDrone2019-MOT-train\annotations"
output_folder = r"F:\ml class project\VisDrone2019-DET-test-challenge\annotations-modified"

process_folder(input_folder, output_folder)


Processing uav0000071_03240_v.txt...
Removing 33 frames (54–86) from object 31 (segment 1–130, 130 frames)
Removing 44 frames (87–130) from object 31 (segment 1–130, 130 frames)
Removing 47 frames (60–106) from object 33 (segment 1–161, 161 frames)
Removing 22 frames (108–129) from object 33 (segment 1–161, 161 frames)
Removing 20 frames (55–74) from object 34 (segment 1–130, 130 frames)
Removing 24 frames (92–115) from object 34 (segment 1–130, 130 frames)
Removing 25 frames (31–55) from object 39 (segment 1–83, 83 frames)
Removing 25 frames (55–79) from object 39 (segment 1–83, 83 frames)
Removing 29 frames (44–72) from object 40 (segment 16–101, 86 frames)
Removing 22 frames (72–93) from object 40 (segment 16–101, 86 frames)
Removing 25 frames (124–148) from object 45 (segment 93–181, 89 frames)
Removing 29 frames (151–179) from object 45 (segment 93–181, 89 frames)
Removing 30 frames (45–74) from object 61 (segment 1–122, 122 frames)
Removing 30 frames (83–112) from object 61 (seg

In [32]:
import os
import pandas as pd

def load_annotation_file(file_path):
    """
    Load a VisDrone annotation file into a DataFrame with proper column names.
    """
    column_names = [
        'frame', 'object_id', 'bbox_left', 'bbox_top', 'bbox_width', 'bbox_height',
        'score', 'object_category', 'truncation', 'occlusion', 'tracklet_id'
    ]
    try:
        df = pd.read_csv(file_path, header=None, names=column_names)
        return df
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

def assign_tracklet_ids(df):
    """
    Assign tracklet IDs by iterating through each object_id and checking frame continuity.
    """
    # Reset tracklet_id column
    df['tracklet_id'] = 0
    global_tracklet_counter = 1  # Unique tracklet IDs within the file

    # Get unique object IDs
    unique_obj_ids = df['object_id'].unique()

    for obj_id in unique_obj_ids:
        # Filter rows for the current object_id and sort by frame
        obj_df = df[df['object_id'] == obj_id].sort_values('frame')
        frames = obj_df['frame'].values
        indices = obj_df.index

        if len(frames) == 0:
            continue

        # Initialize the first tracklet
        tracklet_start_idx = 0

        # Check for gaps in frame sequence
        for i in range(1, len(frames)):
            if frames[i] != frames[i-1] + 1:  # Non-consecutive frame detected
                # Assign tracklet ID to the previous segment
                df.loc[indices[tracklet_start_idx:i], 'tracklet_id'] = global_tracklet_counter
                global_tracklet_counter += 1
                tracklet_start_idx = i

        # Assign tracklet ID to the last (or only) segment
        df.loc[indices[tracklet_start_idx:], 'tracklet_id'] = global_tracklet_counter
        global_tracklet_counter += 1

    return df

def process_folder(input_dir, output_dir):
    """
    Process all VisDrone annotation `.txt` files in a folder and assign tracklet IDs.
    """
    os.makedirs(output_dir, exist_ok=True)
    annotation_files = [f for f in os.listdir(input_dir) if f.endswith('.txt')]

    if not annotation_files:
        print(f"No annotation files found in {input_dir}")
        return

    for file_name in annotation_files:
        input_path = os.path.join(input_dir, file_name)
        df = load_annotation_file(input_path)

        if df is None:
            continue

        print(f"\nProcessing {file_name}...")

        # Assign tracklet IDs
        df = assign_tracklet_ids(df)

        # Save modified file with tracklet IDs
        output_path = os.path.join(output_dir, file_name.replace('.txt', '-corrected.txt'))
        df.to_csv(output_path, header=False, index=False)
        print(f"Saved annotations with tracklet IDs to {output_path}")

# === Example Usage ===
input_folder = r"F:\ml class project\VisDrone2019-DET-test-challenge\annotations-modified"
output_folder = r"F:\ml class project\VisDrone2019-DET-test-challenge\annotations-tracklets-corrected"

process_folder(input_folder, output_folder)


Processing uav0000071_03240_v-modified.txt...
Saved annotations with tracklet IDs to F:\ml class project\VisDrone2019-DET-test-challenge\annotations-tracklets-corrected\uav0000071_03240_v-modified-corrected.txt

Processing uav0000072_04488_v-modified.txt...
Saved annotations with tracklet IDs to F:\ml class project\VisDrone2019-DET-test-challenge\annotations-tracklets-corrected\uav0000072_04488_v-modified-corrected.txt

Processing uav0000072_05448_v-modified.txt...
Saved annotations with tracklet IDs to F:\ml class project\VisDrone2019-DET-test-challenge\annotations-tracklets-corrected\uav0000072_05448_v-modified-corrected.txt

Processing uav0000072_06432_v-modified.txt...
Saved annotations with tracklet IDs to F:\ml class project\VisDrone2019-DET-test-challenge\annotations-tracklets-corrected\uav0000072_06432_v-modified-corrected.txt

Processing uav0000076_00720_v-modified.txt...
Saved annotations with tracklet IDs to F:\ml class project\VisDrone2019-DET-test-challenge\annotations-tr

In [33]:
import os
import pandas as pd

def load_annotation_file(file_path):
    """
    Load a VisDrone annotation file with tracklet IDs into a DataFrame.
    """
    column_names = [
        'frame', 'object_id', 'bbox_left', 'bbox_top', 'bbox_width', 'bbox_height',
        'score', 'object_category', 'truncation', 'occlusion', 'tracklet_id'
    ]
    try:
        df = pd.read_csv(file_path, header=None, names=column_names)
        return df
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

def create_ground_truth(df):
    """
    Create a ground truth DataFrame with object_id and comma-separated tracklet_ids.
    """
    # Group by object_id and join unique tracklet_ids with commas
    gt_df = df.groupby('object_id')['tracklet_id'].unique().reset_index()
    gt_df['tracklet_ids'] = gt_df['tracklet_id'].apply(lambda x: ','.join(map(str, sorted(x))))
    gt_df = gt_df[['object_id', 'tracklet_ids']]
    
    # Debug: Print raw object_id-tracklet_id pairs from input
    print("Raw object_id-tracklet_id pairs from input:")
    raw_pairs = df.groupby('object_id')['tracklet_id'].unique().reset_index()
    for _, row in raw_pairs.iterrows():
        obj_id = row['object_id']
        tracklets = row['tracklet_id'].tolist()
        print(f"object_id {obj_id}: tracklets {tracklets}")
    
    # Debug: Print final ground truth pairs
    print("Ground truth pairs to be saved:")
    for _, row in gt_df.iterrows():
        obj_id = row['object_id']
        tracklets = row['tracklet_ids']
        print(f"object_id {obj_id}: tracklet_ids {tracklets}")
    
    return gt_df

def create_tracklets_only(df):
    """
    Create a DataFrame with object_id removed.
    """
    columns_to_keep = [
        'frame', 'bbox_left', 'bbox_top', 'bbox_width', 'bbox_height',
        'score', 'object_category', 'truncation', 'occlusion', 'tracklet_id'
    ]
    return df[columns_to_keep]

def process_folder(input_dir, output_dir):
    """
    Process all tracklet-assigned annotation files to create ground truth and tracklets-only CSVs.
    """
    os.makedirs(output_dir, exist_ok=True)
    annotation_files = [f for f in os.listdir(input_dir) if f.endswith('.txt')]

    if not annotation_files:
        print(f"No annotation files found in {input_dir}")
        return

    for file_name in annotation_files:
        input_path = os.path.join(input_dir, file_name)
        df = load_annotation_file(input_path)

        if df is None:
            continue

        print(f"\nProcessing {file_name}...")

        # Create ground truth CSV
        gt_df = create_ground_truth(df)
        gt_output_path = os.path.join(output_dir, file_name.replace('.txt', '-gt.csv'))
        gt_df.to_csv(gt_output_path, index=False)
        print(f"Saved ground truth to {gt_output_path}")

        # Create tracklets-only CSV
        tracklets_df = create_tracklets_only(df)
        tracklets_output_path = os.path.join(output_dir, file_name.replace('.txt', '-tracklets-only.csv'))
        tracklets_df.to_csv(tracklets_output_path, index=False)
        print(f"Saved tracklets-only annotations to {tracklets_output_path}")

# === Example Usage ===
input_folder = r"F:\ml class project\VisDrone2019-DET-test-challenge\annotations-tracklets-corrected"
output_folder = r"F:\ml class project\VisDrone2019-DET-test-challenge\annotations-tracklet-merging"

process_folder(input_folder, output_folder)


Processing uav0000071_03240_v-modified-corrected.txt...
Raw object_id-tracklet_id pairs from input:
object_id 26: tracklets [1]
object_id 27: tracklets [2]
object_id 28: tracklets [3]
object_id 30: tracklets [4]
object_id 31: tracklets [5]
object_id 32: tracklets [6]
object_id 33: tracklets [7, 8, 9]
object_id 34: tracklets [10, 11, 12]
object_id 35: tracklets [13]
object_id 36: tracklets [14]
object_id 39: tracklets [15, 16]
object_id 40: tracklets [17, 18]
object_id 41: tracklets [19, 20]
object_id 42: tracklets [21]
object_id 43: tracklets [22]
object_id 44: tracklets [23]
object_id 45: tracklets [24, 25, 26]
object_id 46: tracklets [27]
object_id 47: tracklets [28]
object_id 48: tracklets [29]
object_id 51: tracklets [30]
object_id 61: tracklets [31, 32, 33]
object_id 62: tracklets [34, 35, 36]
object_id 68: tracklets [37]
object_id 69: tracklets [38]
Ground truth pairs to be saved:
object_id 26: tracklet_ids 1
object_id 27: tracklet_ids 2
object_id 28: tracklet_ids 3
object_id 30

In [6]:
import os
import pandas as pd
import numpy as np
import cv2

def load_annotation_file(file_path):
    """Load a tracklet-assigned annotation file into a DataFrame."""
    column_names = ['frame', 'bbox_left', 'bbox_top', 'bbox_width', 'bbox_height',
                   'score', 'object_category', 'truncation', 'occlusion', 'tracklet_id']
    try:
        if file_path.endswith('.txt'):
            df = pd.read_csv(file_path, header=None, names=column_names)
        elif file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
            df = df.rename(columns={col: col.strip() for col in df.columns})
            missing_cols = [col for col in column_names if col not in df.columns]
            if missing_cols:
                print(f"Warning: Missing columns {missing_cols} in {file_path}")
                return None
        return df
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

def extract_annotation_features(tracklet_df):
    """Extract annotation-based features for a single tracklet."""
    features = {}
    center_x = tracklet_df['bbox_left'] + tracklet_df['bbox_width'] / 2
    center_y = tracklet_df['bbox_top'] + tracklet_df['bbox_height'] / 2
    features['mean_center_x'] = center_x.mean()
    features['std_center_x'] = center_x.std()
    features['mean_center_y'] = center_y.mean()
    features['std_center_y'] = center_y.std()
    features['mean_bbox_width'] = tracklet_df['bbox_width'].mean()
    features['std_bbox_width'] = tracklet_df['bbox_width'].std()
    features['mean_bbox_height'] = tracklet_df['bbox_height'].mean()
    features['std_bbox_height'] = tracklet_df['bbox_height'].std()
    features['mean_aspect_ratio'] = (tracklet_df['bbox_width'] / tracklet_df['bbox_height']).mean()
    features['std_aspect_ratio'] = (tracklet_df['bbox_width'] / tracklet_df['bbox_height']).std()
    features['mean_area'] = (tracklet_df['bbox_width'] * tracklet_df['bbox_height']).mean()
    features['std_area'] = (tracklet_df['bbox_width'] * tracklet_df['bbox_height']).std()
    features['start_frame'] = tracklet_df['frame'].min()
    features['end_frame'] = tracklet_df['frame'].max()
    features['frame_duration'] = features['end_frame'] - features['start_frame'] + 1
    velocity_x = np.diff(center_x) / np.diff(tracklet_df['frame'])
    velocity_y = np.diff(center_y) / np.diff(tracklet_df['frame'])
    features['mean_velocity_x'] = np.mean(velocity_x) if len(velocity_x) > 0 else 0
    features['std_velocity_x'] = np.std(velocity_x) if len(velocity_x) > 0 else 0
    features['mean_velocity_y'] = np.mean(velocity_y) if len(velocity_y) > 0 else 0
    features['std_velocity_y'] = np.std(velocity_y) if len(velocity_y) > 0 else 0
    accel_x = np.diff(velocity_x) / np.diff(tracklet_df['frame'].iloc[1:])
    accel_y = np.diff(velocity_y) / np.diff(tracklet_df['frame'].iloc[1:])
    features['mean_acceleration_x'] = np.mean(accel_x) if len(accel_x) > 0 else 0
    features['std_acceleration_x'] = np.std(accel_x) if len(accel_x) > 0 else 0
    features['mean_acceleration_y'] = np.mean(accel_y) if len(accel_y) > 0 else 0
    features['std_acceleration_y'] = np.std(accel_y) if len(accel_y) > 0 else 0
    gaps = np.diff(tracklet_df['frame']) - 1
    features['num_gaps'] = np.sum(gaps > 0)
    features['max_gap_length'] = np.max(gaps) if len(gaps) > 0 else 0
    features['mean_score'] = tracklet_df['score'].mean()
    features['std_score'] = tracklet_df['score'].std()
    features['mode_object_category'] = tracklet_df['object_category'].mode().iloc[0]
    features['mean_truncation'] = tracklet_df['truncation'].mean()
    features['max_truncation'] = tracklet_df['truncation'].max()
    features['mean_occlusion'] = tracklet_df['occlusion'].mean()
    features['max_occlusion'] = tracklet_df['occlusion'].max()
    features['variance_center_x'] = center_x.var()
    features['variance_center_y'] = center_y.var()
    features['cv_bbox_width'] = tracklet_df['bbox_width'].std() / tracklet_df['bbox_width'].mean() if tracklet_df['bbox_width'].mean() > 0 else 0
    features['cv_bbox_height'] = tracklet_df['bbox_height'].std() / tracklet_df['bbox_height'].mean() if tracklet_df['bbox_height'].mean() > 0 else 0
    features['cv_score'] = tracklet_df['score'].std() / tracklet_df['score'].mean() if tracklet_df['score'].mean() > 0 else 0
    features['tracklet_id'] = tracklet_df['tracklet_id'].iloc[0]
    return features

def extract_image_features(tracklet_df, sequence_folder):
    """Extract summarized image-based features (optical flow and color histogram) for a single tracklet."""
    features = {}
    color_hists = []
    flow_magnitudes = []
    flow_directions = []

    # Preload images and sample every 5th frame to reduce workload
    sampled_frames = tracklet_df.iloc[::5]  # Process every 5th frame
    img_cache = {row['frame']: cv2.imread(os.path.join(sequence_folder, f"{row['frame']:07d}.jpg"))
                 for _, row in sampled_frames.iterrows() if cv2.imread(os.path.join(sequence_folder, f"{row['frame']:07d}.jpg")) is not None}

    # Color Histogram with reduced resolution
    for _, row in sampled_frames.iterrows():
        img = img_cache.get(row['frame'])
        if img is None:
            print(f"Warning: Image not found for frame {row['frame']}")
            continue
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        x, y, w, h = int(row['bbox_left']), int(row['bbox_top']), int(row['bbox_width']), int(row['bbox_height'])
        bbox_img = img_rgb[max(0, y):min(img.shape[0], y+h), max(0, x):min(img.shape[1], x+w)]
        if bbox_img.size > 0:
            hsv = cv2.cvtColor(bbox_img, cv2.COLOR_RGB2HSV)
            hist = cv2.calcHist([hsv], [0, 1], None, [25, 30], [0, 180, 0, 256])  # Reduced to 25x30 = 750 bins
            cv2.normalize(hist, hist)
            color_hists.append(hist.flatten())

    if color_hists:
        hist_stats = np.array(color_hists)
        features['color_hist_mean'] = np.mean(hist_stats).item()  # Single value: mean of all histogram values
        features['color_hist_std'] = np.std(hist_stats).item()   # Single value: std of all histogram values
        features['color_hist_min'] = np.min(hist_stats).item()   # Single value: min of all histogram values
        features['color_hist_max'] = np.max(hist_stats).item()   # Single value: max of all histogram values

    # Optical Flow with size adjustment
    for i, (_, row) in enumerate(sampled_frames.iterrows()):
        img = img_cache.get(row['frame'])
        if img is None:
            continue
        if i > 0:
            prev_idx = sampled_frames.index[i-1]
            prev_img = img_cache.get(sampled_frames.at[prev_idx, 'frame'])
            if prev_img is not None:
                # Extract bounding boxes
                prev_x, prev_y, prev_w, prev_h = int(sampled_frames.at[prev_idx, 'bbox_left']), int(sampled_frames.at[prev_idx, 'bbox_top']), int(sampled_frames.at[prev_idx, 'bbox_width']), int(sampled_frames.at[prev_idx, 'bbox_height'])
                x, y, w, h = int(row['bbox_left']), int(row['bbox_top']), int(row['bbox_width']), int(row['bbox_height'])
                prev_bbox = prev_img[max(0, prev_y):min(prev_img.shape[0], prev_y+prev_h), max(0, prev_x):min(prev_img.shape[1], prev_x+prev_w)]
                bbox_img = img[max(0, y):min(img.shape[0], y+h), max(0, x):min(img.shape[1], x+w)]
                if prev_bbox.size == 0 or bbox_img.size == 0:
                    continue

                # Resize to match smaller dimension
                min_height = min(prev_bbox.shape[0], bbox_img.shape[0])
                min_width = min(prev_bbox.shape[1], bbox_img.shape[1])
                prev_bbox_resized = cv2.resize(prev_bbox, (min_width, min_height))
                bbox_img_resized = cv2.resize(bbox_img, (min_width, min_height))

                # Convert to grayscale
                prev_gray = cv2.cvtColor(prev_bbox_resized, cv2.COLOR_BGR2GRAY)
                next_gray = cv2.cvtColor(bbox_img_resized, cv2.COLOR_BGR2GRAY)

                # Compute optical flow
                flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
                flow_magnitudes.append(np.mean(magnitude))
                flow_directions.append(np.mean(np.degrees(angle)))

    if flow_magnitudes:
        features['mean_flow_magnitude'] = np.mean(flow_magnitudes)
        features['std_flow_magnitude'] = np.std(flow_magnitudes)
        features['mean_flow_direction'] = np.mean(flow_directions)
        features['std_flow_direction'] = np.std(flow_directions)

    return features

def process_folder(input_folder, output_folder, sequence_folder):
    """Process all tracklet-assigned annotation files."""
    os.makedirs(output_folder, exist_ok=True)
    annotation_files = [f for f in os.listdir(input_folder) if f.endswith(('-tracklets-only.csv', '-tracklets.txt'))]

    if not annotation_files:
        print(f"No tracklet files found in {input_folder}")
        return

    for file_name in annotation_files:
        input_path = os.path.join(input_folder, file_name)
        df = load_annotation_file(input_path)

        if df is None:
            continue

        print(f"\nProcessing {file_name}...")
        sequence_subfolder = os.path.splitext(file_name.split('-modified')[0])[0]
        sequence_path = os.path.join(sequence_folder, sequence_subfolder)

        if not os.path.exists(sequence_path):
            print(f"Error: Sequence path {sequence_path} does not exist.")
            continue

        # Process all tracklets
        all_features = []
        for tracklet_id in df['tracklet_id'].unique():
            tracklet_df = df[df['tracklet_id'] == tracklet_id].sort_values('frame')
            annotation_features = extract_annotation_features(tracklet_df)
            image_features = extract_image_features(tracklet_df, sequence_path)
            features = {**annotation_features, **image_features}
            all_features.append(features)

        if all_features:
            features_df = pd.DataFrame(all_features)

            base_name = os.path.splitext(file_name)[0]
            output_path = os.path.join(output_folder, f"{base_name}-tracklet-features.csv")
            features_df.to_csv(output_path, index=False)
            print(f"Saved tracklet features to {output_path}")

# === Update with Your Paths ===
input_folder = r"F:\ml class project\VisDrone2019-DET-test-challenge\annotations-tracklet-merging"
output_folder = r"F:\ml class project\VisDrone2019-DET-test-challenge\tracklet-features"
sequence_folder = r"F:\ml class project\VisDrone2019-DET-test-challenge\VisDrone2019-MOT-train\sequences"

process_folder(input_folder, output_folder, sequence_folder)


Processing uav0000071_03240_v-modified-corrected-tracklets-only.csv...
Saved tracklet features to F:\ml class project\VisDrone2019-DET-test-challenge\tracklet-features\uav0000071_03240_v-modified-corrected-tracklets-only-tracklet-features.csv

Processing uav0000072_04488_v-modified-corrected-tracklets-only.csv...
Saved tracklet features to F:\ml class project\VisDrone2019-DET-test-challenge\tracklet-features\uav0000072_04488_v-modified-corrected-tracklets-only-tracklet-features.csv

Processing uav0000072_05448_v-modified-corrected-tracklets-only.csv...
Saved tracklet features to F:\ml class project\VisDrone2019-DET-test-challenge\tracklet-features\uav0000072_05448_v-modified-corrected-tracklets-only-tracklet-features.csv

Processing uav0000072_06432_v-modified-corrected-tracklets-only.csv...
Saved tracklet features to F:\ml class project\VisDrone2019-DET-test-challenge\tracklet-features\uav0000072_06432_v-modified-corrected-tracklets-only-tracklet-features.csv

Processing uav0000076_0

In [1]:
import torch
print(torch.cuda.is_available())           # Should print: True
print(torch.cuda.get_device_name(0))       # Should print: 'NVIDIA GeForce RTX 4060'
print(torch.version.cuda)              # Should print: '12.1'


True
NVIDIA GeForce RTX 4060 Laptop GPU
12.1


In [31]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import precision_recall_fscore_support
from sklearn.impute import SimpleImputer
import networkx as nx
import json

# Define paths
input_folder = r"F:\ml class project\VisDrone2019-DET-test-challenge\tracklet-features"
gt_folder = r"F:\ml class project\VisDrone2019-DET-test-challenge\annotations-tracklet-merging"
output_folder = r"F:\ml class project\VisDrone2019-DET-test-challenge\tracklet-features"
merged_output_folder = r"F:\ml class project\VisDrone2019-DET-test-challenge\merged-tracklets"
os.makedirs(merged_output_folder, exist_ok=True)

def process_file(feature_file, gt_file):
    """
    Process a feature file and its corresponding ground truth file to cluster tracklets and evaluate results.
    
    Args:
        feature_file (str): Path to the tracklet features CSV file.
        gt_file (str): Path to the ground truth CSV file.
    
    Returns:
        tuple: (merged_groups, precision, recall, f1, df_features)
            - merged_groups: Dictionary of cluster labels to tracklet IDs.
            - precision, recall, f1: Evaluation metrics.
            - df_features: Original DataFrame with features.
    """
    # Load data
    df_features = pd.read_csv(feature_file)
    df_ground_truth = pd.read_csv(gt_file)

    # Print available columns for debugging
    print(f"Available columns in {feature_file}: {df_features.columns.tolist()}")

    # Prepare ground truth mapping
    ground_truth_mapping = {}
    for _, row in df_ground_truth.iterrows():
        tracklet_ids_value = row['tracklet_ids']
        tracklet_ids = [int(tid.strip()) for tid in str(tracklet_ids_value).split(',')] if isinstance(tracklet_ids_value, str) else [int(tracklet_ids_value)]
        for tid in tracklet_ids:
            ground_truth_mapping[tid] = row['object_id']

    # Feature selection
    features_to_use = ['mean_center_x', 'mean_center_y', 'mean_velocity_x', 'mean_velocity_y',
                      'color_hist_mean', 'color_hist_std', 'mean_flow_magnitude']
    available_features = [f for f in features_to_use if f in df_features.columns]
    if not available_features:
        print("No usable features found. Skipping processing.")
        return {}, 0.0, 0.0, 0.0, df_features

    # Impute NaN values in selected features
    imputer = SimpleImputer(strategy='constant', fill_value=0)
    X = imputer.fit_transform(df_features[available_features])

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Temporal feature calculation using 'start_frame' and 'end_frame'
    if 'start_frame' in df_features.columns and 'end_frame' in df_features.columns:
        temporal_features = []
        for i in range(len(df_features)):
            for j in range(i + 1, len(df_features)):
                start_i, end_i = df_features.iloc[i]['start_frame'], df_features.iloc[i]['end_frame']
                start_j, end_j = df_features.iloc[j]['start_frame'], df_features.iloc[j]['end_frame']
                intersection = max(0, min(end_i, end_j) - max(start_i, start_j))
                union = max(end_i, end_j) - min(start_i, start_j)
                iou = intersection / union if union > 0 else 0
                temporal_features.append(iou)
        temporal_mean = np.mean(temporal_features) if temporal_features else 0
    else:
        print("Warning: 'start_frame' or 'end_frame' not found. Using default temporal value.")
        temporal_mean = 0

    # Enhance features with temporal mean
    X_enhanced = np.hstack([X_scaled, np.full((len(X), 1), temporal_mean)])

    # Use X_enhanced for clustering
    X_final = X_enhanced

    # Handle case with fewer than 2 samples
    if X_final.shape[0] < 2:
        print(f"Warning: {os.path.basename(feature_file)} has only {X_final.shape[0]} tracklet(s). Skipping clustering.")
        labels = np.array([-1])  # Assign default label for single tracklet
        merged_groups = {}  # No merged groups
        precision, recall, f1 = 0.0, 0.0, 0.0  # Default metrics
    else:
        # Clustering
        clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=0.6, linkage='ward')
        labels = clustering.fit_predict(X_final)

        # Assign merged groups
        merged_groups = {}
        for idx, label in enumerate(labels):
            if label != -1:  # Ignore noise points
                if label not in merged_groups:
                    merged_groups[label] = []
                merged_groups[label].append(df_features['tracklet_id'].iloc[idx])

        # Filter small clusters (optional)
        merged_groups = {k: v for k, v in merged_groups.items() if len(v) >= 2}

        # Evaluation
        true_labels = [ground_truth_mapping.get(tid, -1) for tid in df_features['tracklet_id']]
        pred_labels = [label if label != -1 else -1 for label in labels]

        def create_pairwise_labels(labels_true, labels_pred):
            """
            Generate pairwise labels for ground truth and predictions.
            
            Args:
                labels_true (list): Ground truth object IDs for each tracklet.
                labels_pred (list): Predicted group IDs for each tracklet (-1 for no group).
            
            Returns:
                tuple: (y_true, y_pred) as numpy arrays of pairwise labels.
            """
            n_samples = len(labels_true)
            y_true = []
            y_pred = []
            for i in range(n_samples):
                for j in range(i + 1, n_samples):
                    y_true.append(1 if labels_true[i] == labels_true[j] else 0)
                    y_pred.append(1 if labels_pred[i] == labels_pred[j] and labels_pred[i] != -1 else 0)
            return np.array(y_true), np.array(y_pred)

        y_true, y_pred = create_pairwise_labels(true_labels, pred_labels)
        if len(y_true) > 0:
            precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', zero_division=0)
        else:
            precision, recall, f1 = 0.0, 0.0, 0.0

    return merged_groups, precision, recall, f1, df_features

# Process all files
all_results = []
feature_files = [f for f in os.listdir(input_folder) if f.endswith('-tracklet-features.csv')]
for feature_file_name in feature_files:
    base_name = os.path.splitext(feature_file_name.replace('-tracklet-features', '').replace('-tracklets-only', ''))[0]
    feature_path = os.path.join(input_folder, feature_file_name)
    gt_path = os.path.join(gt_folder, f"{base_name}-gt.csv")

    if os.path.exists(gt_path):
        print(f"\nProcessing {feature_file_name}...")
        merged_groups, precision, recall, f1, df_features = process_file(feature_path, gt_path)
        print(f"Predicted Merged Groups: {merged_groups}")
        print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

        # Save results
        output_path = os.path.join(output_folder, f"{base_name}-tracklet-features_with_predictions.csv")
        df_features.to_csv(output_path, index=False)
        print(f"Saved results to {output_path}")

        # Save merged groups
        merged_output_path = os.path.join(merged_output_folder, f"{base_name}_merged_groups.json")
        merged_groups_converted = {int(k): [int(vv) for vv in v] for k, v in merged_groups.items()}
        with open(merged_output_path, 'w') as f:
            json.dump(merged_groups_converted, f, indent=4)
        print(f"Saved merged groups to {merged_output_path}")

        all_results.append({'file': feature_file_name, 'precision': precision, 'recall': recall, 'f1': f1})
    else:
        print(f"Skipping {feature_file_name}: No matching ground truth file found at {gt_path}")

# Summarize results
if all_results:
    avg_precision = np.mean([r['precision'] for r in all_results])
    avg_recall = np.mean([r['recall'] for r in all_results])
    avg_f1 = np.mean([r['f1'] for r in all_results])
    print("\nSummary Across All Files:")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f}")
    print(f"Average F1-Score: {avg_f1:.4f}")
else:
    print("No files processed.")


Processing uav0000071_03240_v-modified-corrected-tracklets-only-tracklet-features.csv...
Available columns in F:\ml class project\VisDrone2019-DET-test-challenge\tracklet-features\uav0000071_03240_v-modified-corrected-tracklets-only-tracklet-features.csv: ['mean_center_x', 'std_center_x', 'mean_center_y', 'std_center_y', 'mean_bbox_width', 'std_bbox_width', 'mean_bbox_height', 'std_bbox_height', 'mean_aspect_ratio', 'std_aspect_ratio', 'mean_area', 'std_area', 'start_frame', 'end_frame', 'frame_duration', 'mean_velocity_x', 'std_velocity_x', 'mean_velocity_y', 'std_velocity_y', 'mean_acceleration_x', 'std_acceleration_x', 'mean_acceleration_y', 'std_acceleration_y', 'num_gaps', 'max_gap_length', 'mean_score', 'std_score', 'mode_object_category', 'mean_truncation', 'max_truncation', 'mean_occlusion', 'max_occlusion', 'variance_center_x', 'variance_center_y', 'cv_bbox_width', 'cv_bbox_height', 'cv_score', 'tracklet_id', 'color_hist_mean', 'color_hist_std', 'color_hist_min', 'color_hist_